Choose Tutors
https://www.kaggle.com/c/choose-tutors/

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
# paths
RAW_DATA_PATH = Path('./data_raw')
RAW_TRAIN_PATH = RAW_DATA_PATH / 'train.csv'

### 0. Признаки.

In [3]:
train_raw = pd.read_csv(RAW_TRAIN_PATH)
train_raw.info()
train_raw.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   10000 non-null  int64  
 1   age                  10000 non-null  float64
 2   years_of_experience  10000 non-null  float64
 3   lesson_price         10000 non-null  float64
 4   qualification        10000 non-null  float64
 5   physics              10000 non-null  float64
 6   chemistry            10000 non-null  float64
 7   biology              10000 non-null  float64
 8   english              10000 non-null  float64
 9   geography            10000 non-null  float64
 10  history              10000 non-null  float64
 11  mean_exam_points     10000 non-null  float64
 12  choose               10000 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 1015.8 KB


,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


In [4]:
# количественные признаки
NUM_FEATURE_NAMES = ['age', 'years_of_experience', 'lesson_price', 'qualification', 'mean_exam_points']
CAT_FEATURE_NAMES = ['physics', 'chemistry', 'biology', 'english', 'geography','history']
TARGET_NAME = 'choose'

### 1. Пропуски

In [ ]:
train_raw.shape

In [ ]:
def show_empty_rows_count_by_feature_name(df, feature_name):
    empty_rows_count = df[df[feature_name].isna()].shape[0]
    if empty_rows_count:
        print(f'\t{feature_name}: {empty_rows_count}')

In [ ]:
# NO empty values

### 2. Статистика

In [ ]:
train_raw.describe().T

In [ ]:
# гистограммы признаков
train_raw[NUM_FEATURE_NAMES + CAT_FEATURE_NAMES].hist(figsize=(16, 16), bins=14, grid=False);

### 3. Проверить баланс классов

In [5]:
y = train_raw[TARGET_NAME]
y.value_counts()

0    8891
1    1109
Name: choose, dtype: int64

### 4. Предобработка, разбиение train/validation

In [6]:
# Путь до обработанных данных
DATA_PATH = Path('./data')
TRAIN_PATH  = DATA_PATH / 'train.csv'
TEST_PATH = DATA_PATH / 'validation.csv'
TRAIN_TARGET_PATH = DATA_PATH / 'train_target.csv'
TEST_TARGET_PATH = DATA_PATH / 'validation_target.csv'

#### 4.1 Стандартизация

In [7]:
def get_scaled_df(df, num_feature_names):
    
    # стандартизация признака
    def calc_std_feat(x):
        s = (x - x.mean()) / x.std()
        return s
    
    # do-scaling-main
    df_norm = df.copy().astype(np.float64)
    for num_feature in num_feature_names:
        df_norm[num_feature] = calc_std_feat(df[num_feature])
    
    return df_norm

In [8]:
train = get_scaled_df(train_raw, NUM_FEATURE_NAMES)
# train = get_scaled_df(train_raw, NUM_FEATURE_NAMES + CAT_FEATURE_NAMES)
y = train[TARGET_NAME]

In [9]:
train.head(3)

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0.0,-1.345023,-1.117674,0.854466,0.345123,0.0,0.0,0.0,0.0,0.0,0.0,0.703552,0.0
1,1.0,0.771966,0.014262,-0.863783,0.345123,1.0,0.0,1.0,0.0,0.0,1.0,-0.546906,1.0
2,2.0,-2.092195,0.580231,0.090800,-0.906684,1.0,0.0,0.0,0.0,0.0,0.0,0.115101,0.0


#### 4.2 Баланс классов

In [10]:
# based on https://www.kaggle.com/mahhets/standard-logistic-regression
def oversample_df_by_target(df, target_name):
    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()
    
    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1
    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    target_counts = df[target_name].value_counts()
    if (target_counts[minor_class_name] < target_counts[major_class_name]):
        tail_target_count = target_counts[major_class_name] - target_counts[minor_class_name]
        sample = df[df[target_name] == minor_class_name].sample(tail_target_count)
        df = df.append(sample, ignore_index=True)
    
    return df.sample(frac=1) 

In [11]:
train = oversample_df_by_target(train, TARGET_NAME)

In [12]:
y = train[TARGET_NAME]
y.value_counts()

0.0    8891
1.0    8891
Name: choose, dtype: int64

#### 4.3 Разбиение (train/validation)

In [13]:
RANDOM_STATE = 21
from sklearn.model_selection import train_test_split
# разбить train-test
X_train, X_test, y_train, y_test = train_test_split(train, y, 
                                                    shuffle=True,
                                                    test_size=0.3,
                                                    random_state=RANDOM_STATE,
                                                    stratify=y)

In [14]:
display(y_train.value_counts(normalize=True), y_test.value_counts(normalize=True))

0.0    0.50004
1.0    0.49996
Name: choose, dtype: float64

1.0    0.500094
0.0    0.499906
Name: choose, dtype: float64

In [15]:
X_train[['Id'] + NUM_FEATURE_NAMES + CAT_FEATURE_NAMES].to_csv(TRAIN_PATH, index = False, header=True)
X_test[['Id'] + NUM_FEATURE_NAMES + CAT_FEATURE_NAMES].to_csv(TEST_PATH, index = False, header=True)
y_train.to_csv(TRAIN_TARGET_PATH, index = False, header=True)
y_test.to_csv(TEST_TARGET_PATH, index = False, header=True)

### 5. Построение модели

In [16]:
import random
random.seed(RANDOM_STATE)

# N бутстрап-выборок
def get_bootstrap(data, labels, N):
    n_samples = data.shape[1]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        for j in range(n_samples):
            sample_index = random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]

        bootstrap.append((b_data, b_labels))
    return bootstrap

In [17]:
import decision_tree as DTree

def random_forest(data, labels, n_trees):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    i = 0
    for b_data, b_labels in bootstrap:
        i += 1
        print(f'В лес добавляется дерево {i}...')
        forest.append(DTree.build_tree(b_data, b_labels))
        
    return forest

In [18]:
# Функция классификации отдельного объекта

def classify_object(obj, node):

    # Останавливаем рекурсию, если достигли листа
    # Leaf defined at decision_tree
    if isinstance(node, DTree.Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [19]:
# функция формирования предсказания по выборке на одном дереве

def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [20]:
# предсказание голосованием деревьев

def tree_vote(forest, data, isProba=False):

    # добавим предсказания всех деревьев в список
    predictions = []
    j = 0
    for tree in forest:
        j += 1
        print(f'Предсказание дерева {j}...')
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    answers = []
    j = 0
    print(f'Всего предсказаний по объектам:{len(predictions_per_object)}')
    for obj in predictions_per_object:
        if not isProba:
            answers.append(max(set(obj), key=obj.count))
        else:
            # see https://www.kaggle.com/konstantinalbul/choose-tutors-baseline-randomforest
            class_0 = 0
            class_1 = 0
            for itm in obj:
                if itm == 0:
                    class_0 += 1
                else:
                    class_1 += 1
            probability = class_1 / (class_0 + class_1)
            answers.append(probability)
        
    return answers

In [21]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [22]:
# лес n деревьев
def get_n_trees_forest(n_trees):
    train_data = X_train.drop(columns=['Id', TARGET_NAME]).to_numpy()
    test_data = X_test.drop(columns=['Id', TARGET_NAME]).to_numpy()
    train_labels = y_train.to_numpy()
    test_labels = y_test.to_numpy()
    
    my_forest_ = random_forest(train_data, train_labels, n_trees)
    
    # ответы для обучающей выборки
    train_answers_ = tree_vote(my_forest_, train_data)
    # ответы для тестовой
    test_answers_ = tree_vote(my_forest_, test_data)

    train_accuracy_ = accuracy_metric(train_labels, train_answers_)
    print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {train_accuracy_:.3f}')

    test_accuracy_ = accuracy_metric(test_labels, test_answers_)
    print(f'Точность случайного леса из {n_trees} деревьев на тестовой выборке: {test_accuracy_:.3f}')

    return my_forest_, train_answers_, test_answers_, train_accuracy_, test_accuracy_

In [23]:
n_trees = 1500
my_forest_1, train_answers, test_answers, train_accuracy, test_accuracy = get_n_trees_forest(n_trees)

В лес добавляется дерево 1...
В лес добавляется дерево 2...
В лес добавляется дерево 3...
В лес добавляется дерево 4...
В лес добавляется дерево 5...
В лес добавляется дерево 6...
В лес добавляется дерево 7...
В лес добавляется дерево 8...
В лес добавляется дерево 9...
В лес добавляется дерево 10...
В лес добавляется дерево 11...
В лес добавляется дерево 12...
В лес добавляется дерево 13...
В лес добавляется дерево 14...
В лес добавляется дерево 15...
В лес добавляется дерево 16...
В лес добавляется дерево 17...
В лес добавляется дерево 18...
В лес добавляется дерево 19...
В лес добавляется дерево 20...
В лес добавляется дерево 21...
В лес добавляется дерево 22...
В лес добавляется дерево 23...
В лес добавляется дерево 24...
В лес добавляется дерево 25...
В лес добавляется дерево 26...
В лес добавляется дерево 27...
В лес добавляется дерево 28...
В лес добавляется дерево 29...
В лес добавляется дерево 30...
В лес добавляется дерево 31...
В лес добавляется дерево 32...
В лес добавляется

В лес добавляется дерево 262...
В лес добавляется дерево 263...
В лес добавляется дерево 264...
В лес добавляется дерево 265...
В лес добавляется дерево 266...
В лес добавляется дерево 267...
В лес добавляется дерево 268...
В лес добавляется дерево 269...
В лес добавляется дерево 270...
В лес добавляется дерево 271...
В лес добавляется дерево 272...
В лес добавляется дерево 273...
В лес добавляется дерево 274...
В лес добавляется дерево 275...
В лес добавляется дерево 276...
В лес добавляется дерево 277...
В лес добавляется дерево 278...
В лес добавляется дерево 279...
В лес добавляется дерево 280...
В лес добавляется дерево 281...
В лес добавляется дерево 282...
В лес добавляется дерево 283...
В лес добавляется дерево 284...
В лес добавляется дерево 285...
В лес добавляется дерево 286...
В лес добавляется дерево 287...
В лес добавляется дерево 288...
В лес добавляется дерево 289...
В лес добавляется дерево 290...
В лес добавляется дерево 291...
В лес добавляется дерево 292...
В лес до

В лес добавляется дерево 520...
В лес добавляется дерево 521...
В лес добавляется дерево 522...
В лес добавляется дерево 523...
В лес добавляется дерево 524...
В лес добавляется дерево 525...
В лес добавляется дерево 526...
В лес добавляется дерево 527...
В лес добавляется дерево 528...
В лес добавляется дерево 529...
В лес добавляется дерево 530...
В лес добавляется дерево 531...
В лес добавляется дерево 532...
В лес добавляется дерево 533...
В лес добавляется дерево 534...
В лес добавляется дерево 535...
В лес добавляется дерево 536...
В лес добавляется дерево 537...
В лес добавляется дерево 538...
В лес добавляется дерево 539...
В лес добавляется дерево 540...
В лес добавляется дерево 541...
В лес добавляется дерево 542...
В лес добавляется дерево 543...
В лес добавляется дерево 544...
В лес добавляется дерево 545...
В лес добавляется дерево 546...
В лес добавляется дерево 547...
В лес добавляется дерево 548...
В лес добавляется дерево 549...
В лес добавляется дерево 550...
В лес до

В лес добавляется дерево 777...
В лес добавляется дерево 778...
В лес добавляется дерево 779...
В лес добавляется дерево 780...
В лес добавляется дерево 781...
В лес добавляется дерево 782...
В лес добавляется дерево 783...
В лес добавляется дерево 784...
В лес добавляется дерево 785...
В лес добавляется дерево 786...
В лес добавляется дерево 787...
В лес добавляется дерево 788...
В лес добавляется дерево 789...
В лес добавляется дерево 790...
В лес добавляется дерево 791...
В лес добавляется дерево 792...
В лес добавляется дерево 793...
В лес добавляется дерево 794...
В лес добавляется дерево 795...
В лес добавляется дерево 796...
В лес добавляется дерево 797...
В лес добавляется дерево 798...
В лес добавляется дерево 799...
В лес добавляется дерево 800...
В лес добавляется дерево 801...
В лес добавляется дерево 802...
В лес добавляется дерево 803...
В лес добавляется дерево 804...
В лес добавляется дерево 805...
В лес добавляется дерево 806...
В лес добавляется дерево 807...
В лес до

В лес добавляется дерево 1033...
В лес добавляется дерево 1034...
В лес добавляется дерево 1035...
В лес добавляется дерево 1036...
В лес добавляется дерево 1037...
В лес добавляется дерево 1038...
В лес добавляется дерево 1039...
В лес добавляется дерево 1040...
В лес добавляется дерево 1041...
В лес добавляется дерево 1042...
В лес добавляется дерево 1043...
В лес добавляется дерево 1044...
В лес добавляется дерево 1045...
В лес добавляется дерево 1046...
В лес добавляется дерево 1047...
В лес добавляется дерево 1048...
В лес добавляется дерево 1049...
В лес добавляется дерево 1050...
В лес добавляется дерево 1051...
В лес добавляется дерево 1052...
В лес добавляется дерево 1053...
В лес добавляется дерево 1054...
В лес добавляется дерево 1055...
В лес добавляется дерево 1056...
В лес добавляется дерево 1057...
В лес добавляется дерево 1058...
В лес добавляется дерево 1059...
В лес добавляется дерево 1060...
В лес добавляется дерево 1061...
В лес добавляется дерево 1062...
В лес доба

...

Предсказание дерева 68...
Предсказание дерева 69...
Предсказание дерева 70...
Предсказание дерева 71...
Предсказание дерева 72...
Предсказание дерева 73...
Предсказание дерева 74...
Предсказание дерева 75...
Предсказание дерева 76...
Предсказание дерева 77...
Предсказание дерева 78...
Предсказание дерева 79...
Предсказание дерева 80...
Предсказание дерева 81...
Предсказание дерева 82...
Предсказание дерева 83...
Предсказание дерева 84...
Предсказание дерева 85...
Предсказание дерева 86...
Предсказание дерева 87...
Предсказание дерева 88...
Предсказание дерева 89...
Предсказание дерева 90...
Предсказание дерева 91...
Предсказание дерева 92...
Предсказание дерева 93...
Предсказание дерева 94...
Предсказание дерева 95...
Предсказание дерева 96...
Предсказание дерева 97...
Предсказание дерева 98...
Предсказание дерева 99...
Предсказание дерева 100...
Предсказание дерева 101...
Предсказание дерева 102...
Предсказание дерева 103...
Предсказание дерева 104...
Предсказание дерева 105...
Предск

Предсказание дерева 374...
Предсказание дерева 375...
Предсказание дерева 376...
Предсказание дерева 377...
Предсказание дерева 378...
Предсказание дерева 379...
Предсказание дерева 380...
Предсказание дерева 381...
Предсказание дерева 382...
Предсказание дерева 383...
Предсказание дерева 384...
Предсказание дерева 385...
Предсказание дерева 386...
Предсказание дерева 387...
Предсказание дерева 388...
Предсказание дерева 389...
Предсказание дерева 390...
Предсказание дерева 391...
Предсказание дерева 392...
Предсказание дерева 393...
Предсказание дерева 394...
Предсказание дерева 395...
Предсказание дерева 396...
Предсказание дерева 397...
Предсказание дерева 398...
Предсказание дерева 399...
Предсказание дерева 400...
Предсказание дерева 401...
Предсказание дерева 402...
Предсказание дерева 403...
Предсказание дерева 404...
Предсказание дерева 405...
Предсказание дерева 406...
Предсказание дерева 407...
Предсказание дерева 408...
Предсказание дерева 409...
Предсказание дерева 410...
П

Предсказание дерева 681...
Предсказание дерева 682...
Предсказание дерева 683...
Предсказание дерева 684...
Предсказание дерева 685...
Предсказание дерева 686...
Предсказание дерева 687...
Предсказание дерева 688...
Предсказание дерева 689...
Предсказание дерева 690...
Предсказание дерева 691...
Предсказание дерева 692...
Предсказание дерева 693...
Предсказание дерева 694...
Предсказание дерева 695...
Предсказание дерева 696...
Предсказание дерева 697...
Предсказание дерева 698...
Предсказание дерева 699...
Предсказание дерева 700...
Предсказание дерева 701...
Предсказание дерева 702...
Предсказание дерева 703...
Предсказание дерева 704...
Предсказание дерева 705...
Предсказание дерева 706...
Предсказание дерева 707...
Предсказание дерева 708...
Предсказание дерева 709...
Предсказание дерева 710...
Предсказание дерева 711...
Предсказание дерева 712...
Предсказание дерева 713...
Предсказание дерева 714...
Предсказание дерева 715...
Предсказание дерева 716...
Предсказание дерева 717...
П

Предсказание дерева 986...
Предсказание дерева 987...
Предсказание дерева 988...
Предсказание дерева 989...
Предсказание дерева 990...
Предсказание дерева 991...
Предсказание дерева 992...
Предсказание дерева 993...
Предсказание дерева 994...
Предсказание дерева 995...
Предсказание дерева 996...
Предсказание дерева 997...
Предсказание дерева 998...
Предсказание дерева 999...
Предсказание дерева 1000...
Предсказание дерева 1001...
Предсказание дерева 1002...
Предсказание дерева 1003...
Предсказание дерева 1004...
Предсказание дерева 1005...
Предсказание дерева 1006...
Предсказание дерева 1007...
Предсказание дерева 1008...
Предсказание дерева 1009...
Предсказание дерева 1010...
Предсказание дерева 1011...
Предсказание дерева 1012...
Предсказание дерева 1013...
Предсказание дерева 1014...
Предсказание дерева 1015...
Предсказание дерева 1016...
Предсказание дерева 1017...
Предсказание дерева 1018...
Предсказание дерева 1019...
Предсказание дерева 1020...
Предсказание дерева 1021...
Предск

Предсказание дерева 1294...
Предсказание дерева 1295...
Предсказание дерева 1296...
Предсказание дерева 1297...
Предсказание дерева 1298...
Предсказание дерева 1299...
Предсказание дерева 1300...
Предсказание дерева 1301...
Предсказание дерева 1302...
Предсказание дерева 1303...
Предсказание дерева 1304...
Предсказание дерева 1305...
Предсказание дерева 1306...
Предсказание дерева 1307...
Предсказание дерева 1308...
Предсказание дерева 1309...
Предсказание дерева 1310...
Предсказание дерева 1311...
Предсказание дерева 1312...
Предсказание дерева 1313...
Предсказание дерева 1314...
Предсказание дерева 1315...
Предсказание дерева 1316...
Предсказание дерева 1317...
Предсказание дерева 1318...
Предсказание дерева 1319...
Предсказание дерева 1320...
Предсказание дерева 1321...
Предсказание дерева 1322...
Предсказание дерева 1323...
Предсказание дерева 1324...
Предсказание дерева 1325...
Предсказание дерева 1326...
Предсказание дерева 1327...
Предсказание дерева 1328...
Предсказание дерева 

### 6. Решение

In [24]:
RAW_TEST_PATH = RAW_DATA_PATH / 'test.csv'
df_test_raw = pd.read_csv(RAW_TEST_PATH)
df_test = get_scaled_df(df_test_raw, NUM_FEATURE_NAMES)
# df_test = get_scaled_df(df_test_raw, NUM_FEATURE_NAMES + CAT_FEATURE_NAMES)
df_test.head()

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points
0,10000.0,-1.733633,0.008074,1.900372,1.643400,0.0,0.0,0.0,0.0,0.0,0.0,1.904968
1,10001.0,-1.360126,2.266408,0.190191,0.377006,1.0,1.0,0.0,0.0,0.0,0.0,0.504831
2,10002.0,-0.239605,0.008074,-0.949930,-0.889389,0.0,0.0,0.0,0.0,0.0,0.0,-1.411147
3,10003.0,-0.239605,1.137241,2.375422,1.643400,1.0,0.0,0.0,0.0,0.0,0.0,2.052351
4,10004.0,-0.986619,0.572657,-0.569889,0.377006,1.0,0.0,0.0,0.0,0.0,0.0,-0.453158


In [25]:
predict_data = df_test.drop(columns = ['Id']).to_numpy()
predict_data.shape

(10000, 11)

In [26]:
predict_answers = tree_vote(my_forest_1, predict_data, True)

Предсказание дерева 1...
Предсказание дерева 2...
Предсказание дерева 3...
Предсказание дерева 4...
Предсказание дерева 5...
Предсказание дерева 6...
Предсказание дерева 7...
Предсказание дерева 8...
Предсказание дерева 9...
Предсказание дерева 10...
Предсказание дерева 11...
Предсказание дерева 12...
Предсказание дерева 13...
Предсказание дерева 14...
Предсказание дерева 15...
Предсказание дерева 16...
Предсказание дерева 17...
Предсказание дерева 18...
Предсказание дерева 19...
Предсказание дерева 20...
Предсказание дерева 21...
Предсказание дерева 22...
Предсказание дерева 23...
Предсказание дерева 24...
Предсказание дерева 25...
Предсказание дерева 26...
Предсказание дерева 27...
Предсказание дерева 28...
Предсказание дерева 29...
Предсказание дерева 30...
Предсказание дерева 31...
Предсказание дерева 32...
Предсказание дерева 33...
Предсказание дерева 34...
Предсказание дерева 35...
Предсказание дерева 36...
Предсказание дерева 37...
Предсказание дерева 38...
Предсказание дерева 3

Предсказание дерева 311...
Предсказание дерева 312...
Предсказание дерева 313...
Предсказание дерева 314...
Предсказание дерева 315...
Предсказание дерева 316...
Предсказание дерева 317...
Предсказание дерева 318...
Предсказание дерева 319...
Предсказание дерева 320...
Предсказание дерева 321...
Предсказание дерева 322...
Предсказание дерева 323...
Предсказание дерева 324...
Предсказание дерева 325...
Предсказание дерева 326...
Предсказание дерева 327...
Предсказание дерева 328...
Предсказание дерева 329...
Предсказание дерева 330...
Предсказание дерева 331...
Предсказание дерева 332...
Предсказание дерева 333...
Предсказание дерева 334...
Предсказание дерева 335...
Предсказание дерева 336...
Предсказание дерева 337...
Предсказание дерева 338...
Предсказание дерева 339...
Предсказание дерева 340...
Предсказание дерева 341...
Предсказание дерева 342...
Предсказание дерева 343...
Предсказание дерева 344...
Предсказание дерева 345...
Предсказание дерева 346...
Предсказание дерева 347...
П

Предсказание дерева 617...
Предсказание дерева 618...
Предсказание дерева 619...
Предсказание дерева 620...
Предсказание дерева 621...
Предсказание дерева 622...
Предсказание дерева 623...
Предсказание дерева 624...
Предсказание дерева 625...
Предсказание дерева 626...
Предсказание дерева 627...
Предсказание дерева 628...
Предсказание дерева 629...
Предсказание дерева 630...
Предсказание дерева 631...
Предсказание дерева 632...
Предсказание дерева 633...
Предсказание дерева 634...
Предсказание дерева 635...
Предсказание дерева 636...
Предсказание дерева 637...
Предсказание дерева 638...
Предсказание дерева 639...
Предсказание дерева 640...
Предсказание дерева 641...
Предсказание дерева 642...
Предсказание дерева 643...
Предсказание дерева 644...
Предсказание дерева 645...
Предсказание дерева 646...
Предсказание дерева 647...
Предсказание дерева 648...
Предсказание дерева 649...
Предсказание дерева 650...
Предсказание дерева 651...
Предсказание дерева 652...
Предсказание дерева 653...
П

Предсказание дерева 925...
Предсказание дерева 926...
Предсказание дерева 927...
Предсказание дерева 928...
Предсказание дерева 929...
Предсказание дерева 930...
Предсказание дерева 931...
Предсказание дерева 932...
Предсказание дерева 933...
Предсказание дерева 934...
Предсказание дерева 935...
Предсказание дерева 936...
Предсказание дерева 937...
Предсказание дерева 938...
Предсказание дерева 939...
Предсказание дерева 940...
Предсказание дерева 941...
Предсказание дерева 942...
Предсказание дерева 943...
Предсказание дерева 944...
Предсказание дерева 945...
Предсказание дерева 946...
Предсказание дерева 947...
Предсказание дерева 948...
Предсказание дерева 949...
Предсказание дерева 950...
Предсказание дерева 951...
Предсказание дерева 952...
Предсказание дерева 953...
Предсказание дерева 954...
Предсказание дерева 955...
Предсказание дерева 956...
Предсказание дерева 957...
Предсказание дерева 958...
Предсказание дерева 959...
Предсказание дерева 960...
Предсказание дерева 961...
П

Предсказание дерева 1224...
Предсказание дерева 1225...
Предсказание дерева 1226...
Предсказание дерева 1227...
Предсказание дерева 1228...
Предсказание дерева 1229...
Предсказание дерева 1230...
Предсказание дерева 1231...
Предсказание дерева 1232...
Предсказание дерева 1233...
Предсказание дерева 1234...
Предсказание дерева 1235...
Предсказание дерева 1236...
Предсказание дерева 1237...
Предсказание дерева 1238...
Предсказание дерева 1239...
Предсказание дерева 1240...
Предсказание дерева 1241...
Предсказание дерева 1242...
Предсказание дерева 1243...
Предсказание дерева 1244...
Предсказание дерева 1245...
Предсказание дерева 1246...
Предсказание дерева 1247...
Предсказание дерева 1248...
Предсказание дерева 1249...
Предсказание дерева 1250...
Предсказание дерева 1251...
Предсказание дерева 1252...
Предсказание дерева 1253...
Предсказание дерева 1254...
Предсказание дерева 1255...
Предсказание дерева 1256...
Предсказание дерева 1257...
Предсказание дерева 1258...
Предсказание дерева 

In [27]:
preds_final = pd.DataFrame()
preds_final['Id'] = df_test_raw['Id'].copy()
preds_final['choose'] = predict_answers
preds_final.to_csv('./predictions.csv', index=False, encoding='utf-8', sep=',')